在某些情况下，multiprocessing 是 threading 的一个替代者，用于编写需要利用多个 CPU 来规避由于 Python 的全局解释器锁带来的计算瓶颈问题的场合。

In [2]:
import multiprocessing
import time


def slow_worker():
    print('Starting worker')
    time.sleep(0.1)
    print('Finished worker')



p = multiprocessing.Process(target=slow_worker)
print('BEFORE:', p, p.is_alive())

p.start()
print('DURING:', p, p.is_alive())

p.terminate()  # 终止进程，及其子进程
print('TERMINATED:', p, p.is_alive())

p.join()
print('JOINED:', p, p.is_alive())

BEFORE: <Process(Process-3, initial)> False
DURING: <Process(Process-3, started)> True
TERMINATED: <Process(Process-3, started)> True
JOINED: <Process(Process-3, stopped[SIGTERM])> False


In [5]:
import sys

def exit_error():
    sys.exit(1)
    
def exit_ok():
    return 

def return_value():
    return 1

def raises():
    raise RuntimeError('There was an error')
    
def terminated():
    time.sleep(3)
    
jobs = []
funcs = [exit_error, exit_ok, return_value, raises, terminated]
for f in funcs:
    print('Starting process for ', f.__name__)
    j = multiprocessing.Process(target=f, name=f.__name__)
    jobs.append(j)
    j.start()
    
jobs[-1].terminate()
for j in jobs:
    j.join()
    print(f"{j.name:>15} exitcode={j.exitcode}")

Starting process for  exit_error
Starting process for  exit_ok
Starting process for  return_value
Starting process for  raises
Starting process for  terminated


Process raises:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-a4b3dc014785>", line 13, in raises
    raise RuntimeError('There was an error')
RuntimeError: There was an error


     exit_error exitcode=1
        exit_ok exitcode=0
   return_value exitcode=0
         raises exitcode=1
     terminated exitcode=-15


In [6]:
import logging
def worker():
    print('doing some work')
    sys.stdout.flush()
    
multiprocessing.log_to_stderr(logging.DEBUG)
p = multiprocessing.Process(target=worker)
p.start()
p.join()

[INFO/Process-19] child process calling self.run()


doing some work


[INFO/Process-19] process shutting down
[DEBUG/Process-19] running all "atexit" finalizers with priority >= 0
[DEBUG/Process-19] running the remaining "atexit" finalizers
[INFO/Process-19] process exiting with exitcode 0
